<a href="https://colab.research.google.com/github/nidhics/AI_ML_EX/blob/main/RAG_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-community langchainhub chromadb langchain

In [ ]:
#!pip install -q -U google-genai

In [ ]:
# First uninstall conflicting versions
#!pip uninstall -y google-ai-generativelanguage google-generativeai

# Reinstall with correct versions
#!pip install google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15
#!pip install langchain-google-genai


In [ ]:
#!pip install sentence-transformers

In [ ]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader= WebBaseLoader(web_path=["https://www.educosys.com/course/genai"])
docs=loader.load()
print(docs)

[Document(metadata={'source': 'https://www.educosys.com/course/genai', 'title': 'Hands-on Generative AI Course', 'description': 'Hands-on Generative AI Course', 'language': 'en'}, page_content="Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Ongoing LIVE CourseHands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AIJoin Anytime!Get LifeTime AccessAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Generative Models Discriminative and Generative models Generative Adversarial Netwo

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
spilts=text_splitter.split_documents(docs)
print(spilts[0])
print(len(spilts))

page_content='Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Ongoing LIVE CourseHands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AIJoin Anytime!Get LifeTime AccessAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Generative Models Discriminative and Generative models Generative Adversarial Networks (GANs) Variational Autoencoders (VAEs) Probabilistic Data Generation Using VAEs Four Mini Projects using TensorFlow Metrics Visualization using TensorBoard Mini Project - Imple

In [ ]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=spilts, embedding=embeddings)

print(vectorstore._collection.count())
print(vectorstore._collection.get())



30
{'ids': ['a5661f65-1cc2-4986-9c7e-99538de2a1d0', '08edf1c0-8ab8-47b4-911b-325508e788fb', 'c9c3981c-4ccd-4591-bd3b-d79343d1ee94', '2fc70f94-c097-4745-a5a9-398fcbc1cc7a', '85a31ee3-a127-45de-92a8-aa9c8a1b75de', 'fa8f0281-18f9-4045-9a6e-3fe47ab978e1', 'f0e81172-8c77-48a8-854b-41376181eb61', '618bf73c-4761-4dff-8706-b0a5f2300ff5', 'daf4d0ee-baff-4b29-98f4-0adb95092899', 'a9b1e972-bebe-4a5f-8378-05e4c15cd6d3', 'f10f3e61-44f5-4d6b-834c-09ca5cfdb1e3', '5424e610-7f05-4c39-82e5-ab37a641d162', '7ed62bbd-b7a5-45ff-9bac-8a2a31088866', '81394cde-4f8b-44d7-b82a-49b51db1472b', 'b4e02ba9-eacc-42af-9a28-4c870049d2ec', '88c06652-c106-419d-96c3-d48dd389ab68', 'edeb46c6-95d7-4c86-885b-7f1216c93924', '1a16e329-dd4e-450f-b5f6-d0f69f320656', 'e3feeb5f-3d65-41cf-a70d-01ababc7abba', 'f2a44fcb-35b3-49ce-982f-6098194d3bc4', 'a19fe672-4098-45da-aede-b9c5f9be25f1', '3f792509-af11-4786-8849-722b64e146f0', 'b1781a10-748d-4e81-9457-6f2866d3d93d', 'b2853e42-5c65-49cd-a34e-4b83661b88c4', '3a4dcda2-4f5d-492f-af41-f8e

In [ ]:
print("\n collection",vectorstore._collection.get(ids=['a5661f65-1cc2-4986-9c7e-99538de2a1d0'], include=['embeddings',"documents"]))

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain import hub
# Initialize Gemini as the LLM
llm = GoogleGenerativeAI(model="gemini-1.5-flash")
#Create retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
prompt=hub.pull("rlm/rag-prompt")


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [25]:
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
# rag_chain=({"context":retriever,"question":RunnablePassthrough()} | prompt | llm | StrOutputParser())
# rag_chain.invoke("what is the capital of france?")

template = """
You are a helpful assistant. Use the following context to answer the question.
If you don't know the answer, say you don't know.

Context:
{context}

Question:
{question}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# 9. Build RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# 10. Ask a question
query = "What is the main idea of the documents?"
answer = qa_chain.run(query)

print("Q:", query)
print("A:", answer)

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7b1570a27d40>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 126, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 99, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 338, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 263, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:/

KeyboardInterrupt: 